In [ ]:
from osgeo import gdal
import numpy as np
import os
from tqdm import tqdm

In [5]:
import os
data_dir = "data/images/"
f = list(filter(lambda filename: filename.endswith('.img'), os.listdir(data_dir)))
f

['2010-05-01.img',
 '2010-05-02.img',
 '2010-05-03.img',
 '2010-07-01.img',
 '2010-07-02.img',
 '2010-07-03.img',
 '2010-09-01.img',
 '2010-09-02.img',
 '2010-09-03.img',
 'lccswm2010_4.img',
 'priv_mod_v10_2011layer1.img',
 'priv_mod_v10_2011layer2.img']

In [22]:
data_dir = "data/images/"

sample_filename = "lccswm2010_4.img"
features_filenames = [
    "priv_mod_v10_2011layer1.img",
    "priv_mod_v10_2011layer2.img",
    "2010-05-01.img",
    "2010-05-02.img",
    "2010-05-03.img",
    "2010-07-01.img",
    "2010-07-02.img",
    "2010-07-03.img",
    "2010-09-01.img",
    "2010-09-02.img",
    "2010-09-03.img",
]
output_filename = "lc_map.img"

In [33]:
def get_sample_size(sample_filename: str) -> int:
    sample_size = 0
    sample_img = gdal.Open(sample_filename, gdal.GA_ReadOnly)
    sample_band = sample_img.GetRasterBand(1)
    size_x = sample_img.RasterXSize
    size_y = sample_img.RasterYSize

    for y in tqdm(range(size_y)):
        vals = sample_band.ReadAsArray(0, y, size_x, 1)
        nonzero_vals = np.count_nonzero(vals)
        sample_size += nonzero_vals

    return sample_size

In [37]:
sample_path = os.path.join(data_dir, sample_filename)
sample_size = get_sample_size(sample_filename=sample_path)
print(f"sample size: {sample_size}")

100%|██████████| 24349/24349 [00:02<00:00, 11557.76it/s]


sample size: 74029669


In [20]:
from time import sleep

text = ""
for i in tqdm(range(100)):
    sleep(0.25)

100%|██████████| 100/100 [00:26<00:00,  3.83it/s]


In [ ]:
def img_to_nparray():
    ...